In [ ]:
!pip install --upgrade tfds-nightly
!pip install --upgrade tensorflow tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 42.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existi

In [ ]:
import time
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# EMNIST 데이터셋 로드
ds_builder = tfds.builder("emnist/balanced")
ds_builder.download_and_prepare()
ds_info = ds_builder.info
dataset = ds_builder.as_data_source()

# Label Mapping
label_mapping = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
    'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
    'U', 'V', 'W', 'X', 'Y', 'Z',
    'a', 'b', 'd', 'e', 'f', 'g', 'h', 'n', 'q', 'r', 't'
]

# 이미지 크기 조정 및 정규화
def preprocess(image, label):
    image = tf.image.resize(image, (32, 32))    # LeNet-5는 32x32 입력 사용
    image = tf.image.grayscale_to_rgb(image)    # 그레이스케일 이미지를 RGB로 변환
    image = image / 255.0                       # 정규화
    return image, label

# 데이터 생성 (train, test)
X = []
y = []
for example in dataset['train']:
    X.append(example['image'])
    y.append(example['label'])

X_test = []
y_test = []
for example in dataset['test']:
    X_test.append(example['image'])
    y_test.append(example['label'])

X = np.array(X)
y = np.array(y)

X_test = np.array(X_test)
y_test = np.array(y_test)


# dataset 분할 (train:validation = 5:1), test dataset은 emnist dataset에 있는 것으로 사용
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=1/6, random_state=42)

# 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=47)
y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=47)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=47)

# 데이터셋 구성 및 배치 설정
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).map(preprocess).shuffle(1000).batch(32)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).map(preprocess).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(preprocess).batch(32)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/112800 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/balanced/incomplete.LY0CAW_3.1.0/emnist-train.array_record*...:   0…

Generating test examples...:   0%|          | 0/18800 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/balanced/incomplete.LY0CAW_3.1.0/emnist-test.array_record*...:   0%…

Dataset emnist downloaded and prepared to /root/tensorflow_datasets/emnist/balanced/3.1.0. Subsequent calls will reuse this data.


In [ ]:
%load_ext tensorboard

from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, Dropout
from tensorflow.keras.layers import LeakyReLU

tf.random.set_seed(42)
np.random.seed(42)

**최적화 할때 hyperparameter 바꿀때마다 Class, model 이름 새로 정의해서 학습**

*   Activation_Function, Optimizer, Learning_rate 변경 (Epoch는 유동적으로 변경)
    *   Activation_Function : relu, tanh, leakyrelu, elu, selu
    *   Optimizer
    *   Learning_rate : 1e-2, 1e-3, 1e-4 (1e-5)

*   model.summary() 전까지 수정

*   각 코드 셀 처음에 변경사항 Comment 달아주세요


In [ ]:
# LeNet-5 모델 정의
# activation function : relu
# optimizer : Adam, learning rate = 1e-3(0.001) 고정

input_shape = (32,32,3)
num_classes = 47

Epochs = 50
Learning_rate = 1e-3
Activation_Function = 'relu'
Optimizer = Adam(learning_rate=Learning_rate)

class LeNet5_relu(Sequential):   # Class 이름
  def __init__(self, input_shape, num_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, input_shape=input_shape, padding="same"))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2),padding='valid'))
    self.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    self.add(Conv2D(120, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(Flatten())
    self.add(Dense(84, activation=Activation_Function))
    self.add(Dropout(0.5))
    self.add(Dense(num_classes, activation='softmax'))

# 모델 생성
model_Relu = LeNet5_relu(input_shape,num_classes) # 모델 이름
model = model_Relu

model.summary()

# 모델 컴파일
model.compile(optimizer=Optimizer, loss=categorical_crossentropy, metrics=['accuracy'])

# 모델 학습
start_time = time.time()  # 학습 시작 시간
history = model.fit(train_dataset, epochs=Epochs, validation_data=valid_dataset)
training_time = time.time() - start_time  # 학습 시간 계산

# 학습 결과 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 모델 평가
start_time = time.time()  # 추론 시작 시간
test_loss, test_accuracy = model.evaluate(test_dataset)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(X_test)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
print(f"훈련시간 : {training_time/60:.2f} minutes.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")


In [ ]:
# LeNet-5 모델 정의
# activation function : tanh
# optimizer : Adam, learning rate = 1e-3(0.001) 고정

input_shape = (32,32,3)
num_classes = 47

Epochs = 50
Learning_rate = 1e-3
Activation_Function = 'tanh'
Optimizer = Adam(learning_rate=Learning_rate)

class LeNet5_tanh(Sequential):   # Class 이름
  def __init__(self, input_shape, num_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, input_shape=input_shape, padding="same"))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2),padding='valid'))
    self.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    self.add(Conv2D(120, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(Flatten())
    self.add(Dense(84, activation=Activation_Function))
    self.add(Dropout(0.5))
    self.add(Dense(num_classes, activation='softmax'))

# 모델 생성
model_Tanh = LeNet5_tanh(input_shape,num_classes) # 모델 이름
model = model_Tanh

model.summary()

# 모델 컴파일
model.compile(optimizer=Optimizer, loss=categorical_crossentropy, metrics=['accuracy'])

# 모델 학습
start_time = time.time()  # 학습 시작 시간
history = model.fit(train_dataset, epochs=Epochs, validation_data=valid_dataset)
training_time = time.time() - start_time  # 학습 시간 계산

# 학습 결과 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 모델 평가
start_time = time.time()  # 추론 시작 시간
test_loss, test_accuracy = model.evaluate(test_dataset)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(X_test)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
print(f"훈련시간 : {training_time/60:.2f} minutes.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")


In [ ]:
# LeNet-5 모델 정의
# activation function : LeakyReLU
# optimizer : Adam, learning rate = 1e-3(0.001) 고정

input_shape = (32,32,3)
num_classes = 47

Epochs = 50
Learning_rate = 1e-3
Activation_Function = 'LeakyReLU'
Optimizer = Adam(learning_rate=Learning_rate)

class LeNet5_LeakyReLU(Sequential):
  def __init__(self, input_shape, num_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, input_shape=input_shape, padding="same"))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2),padding='valid'))
    self.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    self.add(Conv2D(120, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(Flatten())
    self.add(Dense(84, activation=Activation_Function))
    self.add(Dropout(0.5))
    self.add(Dense(num_classes, activation='softmax'))

# 모델 생성
model_LeakyReLU = LeNet5_LeakyReLU(input_shape,num_classes)
model = model_LeakyReLU

model.summary()

# 모델 컴파일
model.compile(optimizer=Optimizer, loss=categorical_crossentropy, metrics=['accuracy'])

# 모델 학습
start_time = time.time()  # 학습 시작 시간
history = model.fit(train_dataset, epochs=20, validation_data=valid_dataset)
training_time = time.time() - start_time  # 학습 시간 계산

# 학습 결과 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 모델 평가
start_time = time.time()  # 추론 시작 시간
test_loss, test_accuracy = model.evaluate(test_dataset)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(X_test)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
print(f"훈련시간 : {training_time/60:.2f} minutes.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")


In [ ]:
# LeNet-5 모델 정의
# activation function : ELU
# optimizer : Adam, learning rate = 1e-3(0.001) 고정

input_shape = (32,32,3)
num_classes = 47

Epochs = 50
Learning_rate = 1e-3
Activation_Function = 'elu'
Optimizer = Adam(learning_rate=Learning_rate)

class LeNet5_ELU(Sequential):
  def __init__(self, input_shape, num_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, input_shape=input_shape, padding="same"))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2),padding='valid'))
    self.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    self.add(Conv2D(120, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(Flatten())
    self.add(Dense(84, activation=Activation_Function))
    self.add(Dropout(0.5))
    self.add(Dense(num_classes, activation='softmax'))

# 모델 생성
model_ELU = LeNet5_ELU(input_shape,num_classes)
model = model_ELU

model.summary()

# 모델 컴파일
model.compile(optimizer=Optimizer, loss=categorical_crossentropy, metrics=['accuracy'])

# 모델 학습
start_time = time.time()  # 학습 시작 시간
history = model.fit(train_dataset, epochs=20, validation_data=valid_dataset)
training_time = time.time() - start_time  # 학습 시간 계산

# 학습 결과 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 모델 평가
start_time = time.time()  # 추론 시작 시간
test_loss, test_accuracy = model.evaluate(test_dataset)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(X_test)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
print(f"훈련시간 : {training_time/60:.2f} minutes.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")


In [ ]:
# LeNet-5 모델 정의
# activation function : SELU
# optimizer : Adam, learning rate = 1e-3(0.001) 고정

input_shape = (32,32,3)
num_classes = 47

Epochs = 50
Learning_rate = 1e-3
Activation_Function = 'selu'
Optimizer = Adam(learning_rate=Learning_rate)

class LeNet5_SELU(Sequential):
  def __init__(self, input_shape, num_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, input_shape=input_shape, padding="same"))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2),padding='valid'))
    self.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    self.add(Conv2D(120, kernel_size=(5,5), strides=(1,1), activation=Activation_Function, padding='valid'))
    self.add(Flatten())
    self.add(Dense(84, activation=Activation_Function))
    self.add(Dropout(0.5))
    self.add(Dense(num_classes, activation='softmax'))

# 모델 생성
model_SELU = LeNet5_SELU(input_shape,num_classes)
model = model_SELU

model.summary()



# 모델 컴파일
model.compile(optimizer=Optimizer, loss=categorical_crossentropy, metrics=['accuracy'])

# 모델 학습
start_time = time.time()  # 학습 시작 시간
history = model.fit(train_dataset, epochs=20, validation_data=valid_dataset)
training_time = time.time() - start_time  # 학습 시간 계산

# 학습 결과 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 모델 평가
start_time = time.time()  # 추론 시작 시간
test_loss, test_accuracy = model.evaluate(test_dataset)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(X_test)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
print(f"훈련시간 : {training_time/60:.2f} minutes.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")
